## Chanel Twitter

In [33]:
import pandas as pd
chanel_twitt = pd.read_csv("C:/Users/a.tekiouk/Sujet_2/Sujet_2/DATA/twitt_chanel.csv",sep=";", error_bad_lines=False)
chanel_t = chanel_twitt[chanel_twitt['language'] == 'en']
pub_t = chanel_twitt[chanel_twitt['language'] == 'en']['publication_time']

C:\Users\a.tekiouk\AppData\Local\Temp\ipykernel_31804\2998298859.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  chanel_twitt = pd.read_csv("C:/Users/a.tekiouk/Sujet_2/Sujet_2/DATA/twitt_chanel.csv",sep=";", error_bad_lines=False)


#### Ajout des token 'hashtag' et 'arobase' dans la pipe

In [22]:
import spacy

nlp = spacy.load("en_core_web_sm")

from spacymoji import Emoji


nlp.add_pipe("emoji", first=True)

In [23]:
from spacy.language import Language

@Language.component("at")
def hashtag_pipe(doc):
    merged_at = False
    while True:
        for token in doc:
            if token.text == '@':
                if token.head is not None:
                    start_index = token.i
                    end_index = start_index + 1
                    with doc.retokenize() as retokenizer:
                        retokenizer.merge(doc[start_index:end_index+1])
                        merged_at = True
                        break
        if not merged_at:
            break
        merged_at = False
    return doc



In [24]:
nlp.add_pipe("at", first=True)

<function __main__.hashtag_pipe(doc)>

In [25]:
from spacy.tokens import Token
hashtag_getter = lambda token: token.text[0] in ("@")
Token.set_extension("is_at", getter=hashtag_getter, force=True)

In [26]:
@Language.component("hashtag")
def hashtag_pipe(doc):
    merged_hashtag = False
    while True:
        for token in doc:
            if token.text == '#':
                if token.head is not None:
                    start_index = token.i
                    end_index = start_index + 1
                    with doc.retokenize() as retokenizer:
                        retokenizer.merge(doc[start_index:end_index+1])
                        merged_hashtag = True
                        break
        if not merged_hashtag:
            break
        merged_hashtag = False
    return doc

In [27]:
nlp.add_pipe("hashtag", first=True)

<function __main__.hashtag_pipe(doc)>

In [28]:
hashtag_getter = lambda token: token.text[0] in ("#")
Token.set_extension("is_hashtag", getter=hashtag_getter, force=True)

TOP hasthags TOP emojis: 

In [10]:
import numpy as np
def top_hashtags(text, N, top):
    hashtag = []
    index_vector = np.random.randint(len(text), size=N)
    for i in index_vector:
        doc = nlp(text[i])
        for token in doc:
            if token._.is_hashtag:
                hashtag.append(token.text)
    Hashtag_corpus = pd.DataFrame()
    Hashtag_corpus['hashtag'] = hashtag
    Hashtag_corpus2 = Hashtag_corpus.groupby("hashtag").size()
    print(Hashtag_corpus2.nlargest(top).tail(top))

In [11]:
def top_emojis(text, N, top):
    emoji = []
    index_vector = np.random.randint(len(text), size=N)
    for i in index_vector:
        doc = nlp(text[i])
        for token in doc:
            if token._.is_emoji:
                emoji.append(token.text)
    Emoji_corpus = pd.DataFrame()
    Emoji_corpus['emoji'] = emoji
    Emoji_corpus2 = Emoji_corpus.groupby("emoji").size()
    print(Emoji_corpus2.nlargest(top).tail(top))

In [19]:
top_hashtags(chanel_t['text'].tolist(),1000,15)

hashtag
#JENNIE            62
#Blackpink         15
#Jennie            13
#gdragon           10
#chanel             8
#Poshmark           7
#shopmycloset       7
#VisibleWomen       6
#fashion            5
#Chanel             4
#Ridiculousness     4
#angelawhite        4
#b                  4
#beauty             4
#cosme              4
dtype: int64


In [30]:
top_emojis(chanel_t['text'].tolist(), 1000, 15)

emoji
🅾     92
😭     79
📍     59
🔐     54
🇰🇷    32
😍     27
🔹     24
❤️    19
👇     18
🤭     18
🥰     16
🖤     11
😂     11
🤔     10
✨      9
dtype: int64


### Constitution d'un corpus afin de définir les junk post

In [41]:
import re

def clean_hashtag(t): #--text
    hashtag_pattern= re.compile("#[A-Za-z0-9_]+")
    return re.sub(hashtag_pattern,"", t) #On supprime tout les types de #
def clean_at(t):
    at_pattern= re.compile("@[A-Za-z0-9_]+")
    return re.sub(at_pattern,"", t)


def del_double(txt,publication_time,s,method): # --text --liste des date et heure de publication pour chaque élément de la liste --seuil (s dans [0;1] ou [0;100] selon la méthode) --méthode
    t = txt.copy()
    distance = method #initialisiation de la fonction de distance
    i = 0
    r = len(t)
    while(i<r):
        r = len(t)
        j=i+1
        while(j<r):
            if(distance(clean_at(clean_hashtag(t[i])).strip(),clean_at(clean_hashtag(t[j])).strip()) <= s ): # Si la distance entre les deux élemens de la liste inf à seuil
                if(publication_time[i]<publication_time[j]):
                    del t[j] #delete
                    r = len(t) #on actualise la taille de la listes
                else:
                    del t[i]
                    r = len(t) #on actualise la taille de la listes
            else:
                j+=1
        i+=1
    return t
#passer en paramètre l'objet distance

In [42]:
import textdistance

dist = textdistance.levenshtein.normalized_distance


In [46]:
import random
chanel_t_junk_valid = random.choices(chanel_t['text'].tolist(), k=300)
chanel_t_junk_valid = del_double(chanel_t_junk_valid, pub_t.tolist(),0.6,dist)

In [48]:
pd.DataFrame(chanel_t_junk_valid).to_csv("C:/Users/a.tekiouk/Sujet_2/Sujet_2/DATA/chanel_t_junk_valid.csv")

Exemple de requête twitter

`to:NASA porque -argentina lang:es`